# Processing data
### Get the cheapest gas station near to a given address
<img src="../img/map_result.png">

In [31]:
import geopandas as gpd
from haversine import Unit, haversine
import folium
from shapely.geometry import Polygon
from shapely import wkt
from geopandas import datasets, GeoDataFrame, read_file, points_from_xy
import pandas as pd
import os
import numpy as np

In [32]:
data_frame = pd.read_csv('../datasets/refactored_dataset.csv')
data_frame.replace(0.00, np.nan, inplace=True)
data_frame.head(20)

,place_id,name,latitude,longitude,regular,premium,diesel
0,2039,"ESTACION DE SERVICIO CALAFIA, S.A. DE C.V.",32.47641,-116.92140,20.99,24.99,23.39
1,2040,"LAS MEJORES ESTACIONES, S.A DE C.V",20.30370,-99.74484,22.81,24.03,NaN
2,2041,"DIAZ GAS, S.A. DE C.V.",31.71947,-106.45140,17.65,20.07,NaN
3,2042,"COMBU-EXPRESS, S.A. DE C.V.",20.71413,-103.30420,22.14,NaN,NaN
4,2043,"PETROMAX, S.A. DE C.V.",26.03787,-98.29977,18.19,NaN,NaN
5,2044,"ESTACION RAEL, S. DE R.L. DE C.V.",32.51342,-117.07150,21.59,24.99,NaN
6,2045,MULTISERVICIOS LA PILARICA SA DE CV,25.75580,-108.96850,24.05,25.11,NaN
7,2046,"PETROMAX, S.A. DE C.V.",25.77202,-100.29180,22.59,NaN,NaN
8,2047,"ESTACION RAEL, S. DE R.L. DE C.V.",31.86546,-116.60790,21.39,24.69,23.19
9,2048,"ESTACION PIRU, S.A. DE C.V.",32.31202,-117.04570,21.69,24.49,NaN


Use Mapbox API in order to obtain the coords (lat/lng) from a place given an address

- [Mapbox geocoding](https://docs.mapbox.com/help/getting-started/geocoding/)


In [41]:
from dotenv import load_dotenv
import requests
load_dotenv()

def get_coords_from_address(address:str):
    address_to_send = f'{address}, Mexico City, Mexico'
    api_key = os.getenv('MAPBOX_API_KEY')
    URL = f'https://api.mapbox.com/geocoding/v5/mapbox.places/{address_to_send}.json?types=address&access_token={api_key}'
    try:
        response = requests.get(URL).json()
        return response['features'][0]['center']
    except:
        print("Not Found")
        return None
    
def calculate_distance(point1, point2, unit):
    unit_base = Unit.MILES
    if unit == 'km':
        unit_base = Unit.KILOMETERS
    elif unit == 'm':
        unit_base = Unit.METERS

    distance = haversine(point1, point2, unit_base, normalize=True)
    return round(distance,2)

def get_gas_stations_nearby(location_point, df, ratio, unit):
    distances  = []
    sources = []

    for i in range(len(df)):
        distances.append(calculate_distance(
            point1=location_point, 
            point2=(df['latitude'][i],df['longitude'][i]), 
            unit=unit
            )
        )

        sources.append(location_point)

    new_df = df.copy()
    new_df['source'] = sources
    new_df['distance'] = distances
    new_df = new_df[new_df['distance']<=ratio]
    new_df = new_df.reset_index()
    new_df = new_df.drop(columns='index')
    return new_df.sort_values(by='distance', ascending=True)

def locate_gas_stations_in_map(df,map_instance,unit,oil_type, icon):
    min_price = df[oil_type].min(skipna=True)
    max_price = df[oil_type].max(skipna=True)
    for i in range(len(df)):
        distance = f"{df['distance'][i]} {unit}"
        price = df[oil_type][i]
        name_station = df['name'][i]

        if price != np.nan:
            if price == min_price:
                color_point = 'blue'
            elif price == max_price:
                color_point = 'orange'
            else:
                color_point = 'yellow'

            
            html = f"""<b>Gas station name:</b> {name_station}<br>
            <b>Oil type:</b> {oil_type}<br>
            <b>Price: </b> ${price} <br>
            <b>Distance: </b> {distance}
            """

            iframe = folium.IFrame(html,figsize=(6,3))
            popup = folium.Popup(iframe)


            folium.Marker(
                location=[float(df['latitude'][i]), float(df['longitude'][i])],
                icon=folium.Icon(color=color_point, icon_color='white', icon=icon),
                prefix='glyphicon',
                popup=popup
            ).add_to(map_instance)


#### Searching a place by address and getting coords

In [34]:

# Searching params
address = 'Zócalo de la ciudad de México'
ratio = 1
unit = 'km'

coords = get_coords_from_address(address=address)
coords.reverse() #lat/lng order
coords

[19.7006015, -101.2301513]

#### Locate the searched place in map via folium

In [36]:
folium_map_instance = folium.Map(coords, zoom_start=15)

# Add circle to map (ratio)
folium.Circle(
    location=coords,
    color='green',
    fill = 'green',
    radius=ratio*1000
).add_to(folium_map_instance)


# Add marker (searched placed)
folium.Marker(
    location=coords,
    icon=folium.Icon(color='black', icon_color='white', icon='home', prefix='glyphicon'),
    popup='<h5>Im here</h5>'
).add_to(folium_map_instance)

# show map up 
folium_map_instance

#### Get the gas stations near the searched location.

In [37]:
# Put gas stations nearby into a Data Frame
gas_station_distances_df = get_gas_stations_nearby(coords,data_frame,ratio,unit)
gas_station_distances_df

,place_id,name,latitude,longitude,regular,premium,diesel,source,distance
0,9428,SERVICIOS LA QUEMADA SA DE CV,19.70302,-101.2301,21.85,NaN,24.95,"[19.7006015, -101.2301513]",0.27
1,11196,MULTISERVICIOS DEL CRUCERO S.A. DE C.V.,19.70517,-101.2249,22.25,NaN,23.69,"[19.7006015, -101.2301513]",0.75


In [38]:
# Getting the geometry from geopandas
gdf_gas_stations_nearby = GeoDataFrame(
    gas_station_distances_df,
    geometry=points_from_xy(
        gas_station_distances_df.longitude,
        gas_station_distances_df.latitude,
    )
)

In [39]:
gdf_gas_stations_nearby

,place_id,name,latitude,longitude,regular,premium,diesel,source,distance,geometry
0,9428,SERVICIOS LA QUEMADA SA DE CV,19.70302,-101.2301,21.85,NaN,24.95,"[19.7006015, -101.2301513]",0.27,POINT (-101.23010 19.70302)
1,11196,MULTISERVICIOS DEL CRUCERO S.A. DE C.V.,19.70517,-101.2249,22.25,NaN,23.69,"[19.7006015, -101.2301513]",0.75,POINT (-101.22490 19.70517)


#### Place the gas stations nearby in the map

- Look for oil type `(regular, premium, diesel)`
- Blue The cheapest
- Yellow: between blue and Orange
- Orange: The most expensive

In [40]:
locate_gas_stations_in_map(
    df=gdf_gas_stations_nearby,
    map_instance=folium_map_instance,
    unit=unit, 
    oil_type='regular',
    icon='usd'
    )

folium_map_instance
